In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/cjlin1/libsvm.git

Cloning into 'libsvm'...
remote: Enumerating objects: 4201, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 4201 (delta 142), reused 119 (delta 119), pack-reused 3969 (from 3)
Receiving objects: 100% (4201/4201), 9.91 MiB | 14.71 MiB/s, done.
Resolving deltas: 100% (2318/2318), done.


In [ ]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


In [31]:
!nvcc -arch=sm_{major}{minor} \
  /content/main.cu \
  /content/cpu_data_loader.cpp \
  /content/cpu_layers.cpp \
  /content/cpu_autoencoder.cpp \
  /content/data_loader.cu \
  /content/gpu_layers.cu \
  /content/gpu_autoencoder.cu \
  /content/model.cu \
  /content/visualization.cu \
  /content/progress_bar.cu \
  /content/timer.cu \
  /content/utils.cu \
  /content/libsvm/svm.cpp \
  -Xcompiler -fopenmp -lgomp \
  -std=c++20 \
  -o main

In [ ]:
#./main [RUN_MODE] [HARDWARE_MODE] [n_batches] [n_epoch] [batch_size] [learning_rate]
!./main phase_2 cpu 1 2

Loading and preprocessing dataset...
Loading training data from /content/drive/MyDrive/@fithcmú/LapTrinhSongSong/data/cifar-10-batches-bin...
  ✓ Loaded batch 1/1
✓ Training data loaded: 10000 samples
  Data range: [0.0000, 1.0000]
Skipping Phase 1: Loading trained Autoencoder and encoding dataset


# Image_Reconstruction

In [ ]:
!nvidia-smi
!nvcc --version

In [ ]:
import os

# Download CIFAR-10 binary dataset
if not os.path.exists('/content/cifar-10-batches-bin'):
    !wget https://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
    !tar -xzvf cifar-10-binary.tar.gz
    print('✓ CIFAR-10 dataset downloaded and extracted')
else:
    print('✓ CIFAR-10 dataset already exists')

# Verify files
!ls -lh /content/cifar-10-batches-bin/

In [ ]:
!mkdir -p /content/output
print('✓ Output directory created')

In [ ]:
from google.colab import files
import zipfile

# Option 1: Upload a zip file containing your project
print('Upload your project as a ZIP file:')
uploaded = files.upload()

# Extract the zip
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/')
        print(f'✓ Extracted {filename}')

# Verify structure
!ls -la /content/

In [ ]:
# Compile with nvcc
!cd /content && nvcc -std=c++20 -arch=sm_75 -O3 \
    -I./include -I./include/cpu \
    -o image_recon.exe \
    image_colab.cu \
    src/data_loader.cu \
    src/cpu/cpu_autoencoder.cu \
    src/cpu/cpu_layers.cu

print('\n✓ Compilation complete')

In [ ]:
!cd /content && ./image_recon.exe

In [ ]:
# Install imagemagick for conversion (if needed)
!apt-get install -y imagemagick

# Convert PPM to PNG for easier viewing
import os
from PIL import Image
import matplotlib.pyplot as plt

output_dir = '/content/output'
K = 8  # Number of samples

# Convert all PPM files to PNG
for i in range(K):
    orig_ppm = f'{output_dir}/orig_{i}.ppm'
    recon_ppm = f'{output_dir}/recon_{i}.ppm'

    if os.path.exists(orig_ppm):
        img = Image.open(orig_ppm)
        img.save(f'{output_dir}/orig_{i}.png')

    if os.path.exists(recon_ppm):
        img = Image.open(recon_ppm)
        img.save(f'{output_dir}/recon_{i}.png')

print('✓ Converted PPM to PNG')

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

K = 8
output_dir = '/content/output'

fig, axes = plt.subplots(2, K, figsize=(20, 5))
fig.suptitle('CIFAR-10 Autoencoder: Original vs Reconstructed', fontsize=16)

for i in range(K):
    # Original
    orig_path = f'{output_dir}/orig_{i}.png'
    if os.path.exists(orig_path):
        img = Image.open(orig_path)
        axes[0, i].imshow(img)
        axes[0, i].axis('off')
        if i == 0:
            axes[0, i].set_title('Original', fontsize=10)

    # Reconstructed
    recon_path = f'{output_dir}/recon_{i}.png'
    if os.path.exists(recon_path):
        img = Image.open(recon_path)
        axes[1, i].imshow(img)
        axes[1, i].axis('off')
        if i == 0:
            axes[1, i].set_title('Reconstructed', fontsize=10)

plt.tight_layout()
plt.show()

print('\n✅ Image reconstruction complete!')

In [ ]:
# Create a zip of all output images
import zipfile

zip_path = '/content/reconstruction_results.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for i in range(K):
        for img_type in ['orig', 'recon']:
            for ext in ['ppm', 'png']:
                img_path = f'{output_dir}/{img_type}_{i}.{ext}'
                if os.path.exists(img_path):
                    zipf.write(img_path, f'{img_type}_{i}.{ext}')

print(f'✓ Created {zip_path}')

# Download the zip file
from google.colab import files
files.download(zip_path)

print('✅ Download started!')